In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
transform_train = transforms.Compose(
    [
     transforms.Resize(32,32), 
     transforms.RandomHorizontalFlip(),  ##좌우 대칭 Data Augmentation 이외에는 성능이 떨어짐.
     #transforms.RandomRotation((-5,5)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
     ]
)

transform_valid = transforms.Compose(
    [transforms.Resize(32,32),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
     ]
)
transform_test = transforms.Compose(
    [transforms.Resize(32,32),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
     ]
)

In [21]:
trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True, 
                                        transform=transform_train)

validset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True, 
                                        transform=transform_valid)

testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform_test)



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [22]:
len(trainset)

50000

In [0]:
index = list(range(len(trainset)))

In [24]:
len(testset)

10000

### TrainSet ValidationSet 9 : 1 비율로 split

In [0]:
tr_len = int(len(trainset) * 0.9)
va_len = int(len(trainset) * 0.1)
# train_data, valid_data = torch.utils.data.random_split(trainset,[tr_len, va_len])

In [0]:
np.random.seed(4321)
np.random.shuffle(index)

In [0]:
train_index, valid_index = index[:tr_len], index[-va_len:]

In [0]:
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)

### Train, Valid index를 뽑아준다음 DataLoader를 만들어준다. 이때, shuffle option은 SubsetRandomSampler와 같이 쓰이지 못하므로 제외시켜준다.

In [0]:
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=4, sampler=train_sampler, num_workers=0
)

valid_loader = torch.utils.data.DataLoader(
    validset, batch_size=4, sampler=valid_sampler, num_workers=2
)

test_loader = torch.utils.data.DataLoader(
    testset, batch_size=4, shuffle=False, num_workers=2
)


### Model Layer

In [30]:
!pip install torchsummary

In [0]:
from torchsummary import summary

### Input 3x32x32 = 3072

In [0]:
class DNN(torch.nn.Module):
  def __init__(self):
    super(DNN, self).__init__()
    self.input_size = 3072
    self.hidden_size = 1024
    self.linear1 = torch.nn.Linear(self.input_size, self.hidden_size)
    self.relu1 = torch.nn.ReLU()
    # self.linear2 = torch.nn.Linear(self.hidden_size, 512)
    # self.relu2 = torch.nn.ReLU()
    self.linear3 = torch.nn.Linear(1024, 256)
    self.relu3 = torch.nn.ReLU()
    # self.linear4 = torch.nn.Linear(256, 128)
    # self.relu4 = torch.nn.ReLU()
    self.linear5 = torch.nn.Linear(256, 64)
    self.relu5 = torch.nn.ReLU()
    self.linear6 = torch.nn.Linear(64, 32)
    self.relu6 = torch.nn.ReLU()
    self.linear7 = torch.nn.Linear(32, 10) #class label 10개.

  def forward(self, input_tensor):
    input_tensor = input_tensor.view(-1,3072)
    linear1 = self.linear1(input_tensor)
    relu1 = self.relu1(linear1)
    # linear2 = self.linear2(relu1)
    # relu2 = self.relu2(linear2)
    linear3 = self.linear3(relu1)
    relu3 = self.relu3(linear3)
    # linear4 = self.linear4(relu3)
    # relu4 = self.relu4(linear4)
    linear5 = self.linear5(relu3)
    relu5 = self.relu5(linear5)
    linear6 = self.linear6(relu5)
    relu6 = self.relu6(linear6)
    linear7 = self.linear7(relu6)
    return linear7



### Params
### Transforms Preprocessing, Parameter Tuning 진행을 해보았지만 
### 단순 MLP 만으로는 한계가 있음. 56% 최대.

In [0]:
model = DNN()
learning_rate = 0.0005
epochs = 14 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.7)
#모멘텀은 신경망의 학습 안정성과 속도를 높여 학습이 잘 하려고 사용. 
#모멘텀 은 다음과 같이 가중치를 갱신할 때 델타 규칙에 모멘텀을 추가로 더함.
#모멘텀을 사용하면 가중치 값이 바로 바뀌지 않고 어느 정도 일정한 방향을 유지하면서 움직이게 됨. 
#또한 가속도처럼 같은 방향으로 더 많이 변화시켜 학습속도를 높여줘 빠른 학습을 하게 됨.

#같은 방향으로 연속으로 가중치가 변화되었으므로 가중치가 더 크게 변경. 
#역으로, 첫 번째와 두 번째의 가중치 방향이 반대이면 변화량이 감소합니다. 
#쉽게 플러스(+) 부호에서 마이너스(-) 부호로 변경되므로 변화량이 적어진다.

### Keras 의 model.summary 와 같은 기능으로 Output Shape과 파라미터를 볼 수 있다. 
### 단, shape을 argument로 넣어줘야 한다. 디버깅 용도로 이용.

In [39]:
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]       3,146,752
              ReLU-2                 [-1, 1024]               0
            Linear-3                  [-1, 256]         262,400
              ReLU-4                  [-1, 256]               0
            Linear-5                   [-1, 64]          16,448
              ReLU-6                   [-1, 64]               0
            Linear-7                   [-1, 32]           2,080
              ReLU-8                   [-1, 32]               0
            Linear-9                   [-1, 10]             330
Total params: 3,428,010
Trainable params: 3,428,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 13.08
Estimated Total Size (MB): 13.11
-------------------------------------

### Train

In [40]:
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for i, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인)
    # 갱신할 변수들에 대한 모든 변화도를 0으로 만듬. 이렇게 하는 이유는
    # 기본적으로 .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고)
    # 누적되기 때문.
    output = model(data)
    loss = criterion(output, target)
    loss.backward()   #역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산
    optimizer.step()  # Optimizer의 step 함수를 호출하면 매개변수가 갱신
    running_loss += loss.item()
    if i % 4000 == 3999:    # print every 2000 mini-batches
      print('Training Loss : [%d, %5d] loss: %.3f' %
        (epoch + 1, i + 1, running_loss / 4000))
      running_loss = 0.0

  #Validation 
  correct = 0
  total = 0
  for i, (data, target) in enumerate(valid_loader, 0):
    data, target = Variable(data), Variable(target)
    output = model(data)
    _, predicted = torch.max(output.data, 1) # 아래 test의 max와 다른 max 이용 방법.
    #torch.max 함수는 텐서의 최대 값이 들어있는 index를 반환함.
    #test에서 설명하지만 max는 value, index를 return. _ 에는 따라서 value가 들어가있음.
    total += target.size(0)
    correct += (predicted == target).sum().item()
  print('#### [%d iteration] Validation Accuracy : [%d %%] ' % (epoch+1, 100*correct/total))  
  if epoch % 1 == 0 or epoch % 9 == 0:
    print('Train loss at {}/{} is {}'.format(epoch+1, epochs, loss.item()))
  


Training Loss : [1,  4000] loss: 2.299
Training Loss : [1,  8000] loss: 2.231
#### [1 iteration] Validation Accuracy : [28 %] 
Train loss at 1/14 is 1.8312065601348877
Training Loss : [2,  4000] loss: 1.903
Training Loss : [2,  8000] loss: 1.812
#### [2 iteration] Validation Accuracy : [38 %] 
Train loss at 2/14 is 1.939528465270996
Training Loss : [3,  4000] loss: 1.673
Training Loss : [3,  8000] loss: 1.626
#### [3 iteration] Validation Accuracy : [43 %] 
Train loss at 3/14 is 1.6026771068572998
Training Loss : [4,  4000] loss: 1.531
Training Loss : [4,  8000] loss: 1.514
#### [4 iteration] Validation Accuracy : [47 %] 
Train loss at 4/14 is 2.1144301891326904
Training Loss : [5,  4000] loss: 1.451
Training Loss : [5,  8000] loss: 1.415
#### [5 iteration] Validation Accuracy : [50 %] 
Train loss at 5/14 is 1.1451464891433716
Training Loss : [6,  4000] loss: 1.371
Training Loss : [6,  8000] loss: 1.354
#### [6 iteration] Validation Accuracy : [52 %] 
Train loss at 6/14 is 1.0423595905

### Test

In [0]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data,target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    #print(output)
    test_loss += criterion(output,target).item()#.data[0]
    pred = output.data.max(1, keepdim=True)[1] #max값은 value, index를 반환한다.
    #따라서. [1] 은 index 즉 어느 것이 확률이 제일 큰지 label 반환
    #max 안의 1 은 어느 방향으로 큰 값을 찾을지 결정.
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    #pred.eq는 안에 값과 같은지 검사.

    test_loss /= len(test_loader.dataset)
  print('Test  {}/{},  Test Accuracy : {}'.format(correct,len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))   

### Result

In [42]:
test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Test  5639/10000,  Test Accuracy : 56.38999938964844
